In [1]:
#remove cell visibility
from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Promijeni vidljivost <a href="javascript:code_toggle()">ovdje</a>.''')
display(tag)

In [2]:
%matplotlib inline
import control
import numpy
import sympy as sym
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt


#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

## Upravljanje glavom čvrstog diska

<img src="Images\EX32-HardDiskHead.PNG" alt="drawing" width="200x200">

Ručica čvrstog diska koja podupire glavu za čitanje rotira se na viskoznom ležaju i pokreće je električni motor; povratna opruga torzijske krutosti $k$ suprotstavlja se njenom kretanju. Ležaj posjeduje lagano viskozno trenje, proporcionalno brzini rotacije glave, s konstantom $b$. Aktuator ručice može generirati okretni moment $T$ linearno proporcionalan struji $I$ koja teče unutar zavojnice: $T = k_iI$. Inercija ručice iznosi $J$. Senzor mjeri kutni položaj ručice $\theta$.


Konstante sustava su:
- $J = 0.001$ kg$\text{m}^2$,
- $k_i = 500$ Nm/A,
- $k = 1$ Nm/rad,
- $b = 0.04$ Nm/(rad/s).

Cilj je dizajnirati regulator za položaj ručice prema sljedećim specifikacijama:
- nema prekoračenja,
- vrijeme smirivanja za 2% pojasa tolerancije manje od 50 milisekundi,
- nulta pogreška u stacionarnom stanju u odzivu na promjenu željenog položaja ručice,
- ulazna struja ne prelazi 5 A u odzivu za ulazni korak od 30 stupnjeva.


Jednadžba suatava je
$$
J\ddot{\theta} = k_iI - b\dot{\theta} - k\theta
$$

Definiranjem vektora stanja kao $x=\begin{bmatrix}x_1 & x_2\end{bmatrix}^T=\begin{bmatrix}\theta & \dot{\theta}\end{bmatrix}^T$ i $u=I$, jednadžbe sustava u matričnoj formi postaju:

\begin{cases}
\dot{x} = \begin{bmatrix} 0 & 1 \\ -k & -b \end{bmatrix}x + \begin{bmatrix} 0 \\ k_i \end{bmatrix}u \\
y = \begin{bmatrix} 1 & 0 \end{bmatrix}x
\end{cases}

Za postizanje nulte pogreške u stacionarnom stanju, sustav proširujemo novim stanjem definiranim kao $\dot{x_3}=y-y_d=x_1-y_d$. Jednadžbe stoga postaju:

\begin{cases}
\dot{x_a} = \begin{bmatrix} 0 & 1 & 0 \\ -k & -b & 0 \\ 1 & 0 & 0 \end{bmatrix}x_a + \begin{bmatrix} 0 & 0 \\ k_i & 0 \\ 0 & -1 \end{bmatrix}\begin{bmatrix} u \\ y_d \end{bmatrix} \\
y = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 0 & 1 \end{bmatrix}x
\end{cases}

Matrica upravljivosti $\mathcal{C}$, uzimajući u obzir samo ulaz $u$, je

In [3]:
A = numpy.matrix('0 1 0; -1 -0.04 0; 1 0 0')
B = numpy.matrix('0 0; 500 0; 0 -1')
C = numpy.matrix('1 0 0; 0 0 1')
D = numpy.matrix('0 0; 0 0')

CM = control.ctrb(A,B[:,0])
display(Markdown(bmatrix(CM)))
# print(numpy.linalg.matrix_rank(CM))

\begin{bmatrix}
  0. & 500. & -20.\\
  500. & -20. & -499.2\\
  0. & 0. & 500.\\
\end{bmatrix}

i ima rang jednak 3, tako da je sustav upravljiv.
Matrica osmotrivosti $\mathcal{O}$ je

In [4]:
OM = control.obsv(A,C)
display(Markdown(bmatrix(OM)))
# print(numpy.linalg.matrix_rank(OM))

\begin{bmatrix}
  1. & 0. & 0.\\
  0. & 0. & 1.\\
  0. & 1. & 0.\\
  1. & 0. & 0.\\
  -1. & -0.04 & 0.\\
  0. & 1. & 0.\\
\end{bmatrix}

i ima rang jednak 3, tako da je sustav osmotriv.


### Dizajn regulatora
#### Dizajn kontrolera

Pojačanje povratne veze stanja dizajniramo uzimajući u obzir samo ulazni signal $u$, dok za referencu koristimo $y_d$. Tranzijentne specifikacije mogu se dobiti ako se dominantni polovi zatvorene petlje postave u $-100.02\pm101i$, a treći pol barem ne sporije, kao na primjer u $-100$. To se postiže matricom pojačanja:
$$
K = \begin{bmatrix} 80.416 & 0.6 & 4041.0 \end{bmatrix}.
$$

#### Dizajn promatrača
Kako bismo imali dobru procjenu stanja i, prema tome, brzu konvergenciju pogreške procjene u nulu, postavljamo polove promatrača u $-151$, $-152$ and $-153$ (brže od polova kontroliranog sustava).
Matrica pojačanja promatrača je:

$$
L = \begin{bmatrix} 300.96 & 3.028e-12 \\
 2.264e+04 & 4.55e-10 \\
 1.0 & 150.0 \end{bmatrix}.
$$

Simulacije su prikazane u nastavku. Imajte na umu da proširivanjem vektora stanja osiguravamo da je pojačanje zatvorene petlje uvijek 1.

### Kako koristiti ovaj interaktivni primjer?
- Pokušajte poboljšati odziv sustava u slučaju male početne pogreške (oko 0,05) u prva dva elementa početnog uvjeta promatrača (parametar X0 est. u nastavku).
- Uvođenje početne pogreške u treću komponentu estimiranog stanja nema smisla jer treće stanje ima dinamiku koju smo sami definirali. Mogli smo razviti i promatrača samo za $x_2$, jer mjerimo $x_1$, a znamo $x_3$.



In [5]:
# Preparatory cell

X0 = numpy.matrix('0.0; 0.0; 0.0')
K = numpy.matrix([8/15,-4.4,-4])
L = numpy.matrix([[300, 3.028E-12],[2.264E+04, 4.55E-10],[1, 150]])

Aw = matrixWidget(3,3)
Aw.setM(A)
Bw = matrixWidget(3,1)
Bw.setM(B)
Cw = matrixWidget(1,3)
Cw.setM(C)
X0w = matrixWidget(3,1)
X0w.setM(X0)
Kw = matrixWidget(1,3)
Kw.setM(K)
Lw = matrixWidget(3,2)
Lw.setM(L)


eig1c = matrixWidget(1,1)
eig2c = matrixWidget(2,1)
eig3c = matrixWidget(1,1)
eig1c.setM(numpy.matrix([-100.])) 
eig2c.setM(numpy.matrix([[-100.02],[-101.]]))
eig3c.setM(numpy.matrix([-100.]))

eig1o = matrixWidget(1,1)
eig2o = matrixWidget(2,1)
eig3o = matrixWidget(1,1)
eig1o.setM(numpy.matrix([-150.])) 
eig2o.setM(numpy.matrix([[-151.],[0.]]))
eig3o.setM(numpy.matrix([-152.]))

In [6]:
# Misc

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

# Define type of method 
selm = widgets.Dropdown(
    options= ['Postavi K i L', 'Postavi svojstvene vrijednosti'],
    value= 'Postavi svojstvene vrijednosti',
    description='',
    disabled=False
)

# Define the number of complex eigenvalues
sele = widgets.Dropdown(
    options= ['0 kompleksnih svojstvenih vrijednosti', '2 kompleksne svojstvene vrijednosti'],
    value= '2 kompleksne svojstvene vrijednosti',
    description='Kompleksne svojstvene vrijednosti:',
    style = {'description_width': 'initial'},
    disabled=False
)

#define type of ipout 
selu = widgets.Dropdown(
    options=['impuls', 'step', 'sinus', 'Pravokutni val'],
    value='step',
    description='Tip referentnog signala:',
    style = {'description_width': 'initial'},
    disabled=False
)
# Define the values of the input
u = widgets.FloatSlider(
    value=30*numpy.pi/180,
    min=0,
    max=45*numpy.pi/180,
    step=0.05,
    description='Referentni signal:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
period = widgets.FloatSlider(
    value=0.5,
    min=0.001,
    max=10,
    step=0.001,
    description='Period: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

gain_w2 = widgets.FloatText(
    value=1.,
    description='',
    disabled=True
)

simTime = widgets.FloatText(
    value=0.5,
    description='',
    disabled=False
)

In [7]:
# Support functions

def eigen_choice(sele):
    if sele == '0 kompleksnih svojstvenih vrijednosti':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = True
        eig1o.children[0].children[0].disabled = False
        eig2o.children[1].children[0].disabled = True
        eig = 0
    if sele == '2 kompleksne svojstvene vrijednosti':
        eig1c.children[0].children[0].disabled = True
        eig2c.children[1].children[0].disabled = False
        eig1o.children[0].children[0].disabled = True
        eig2o.children[1].children[0].disabled = False
        eig = 2
    return eig

def method_choice(selm):
    if selm == 'Postavi K i L':
        method = 1
        sele.disabled = True
    if selm == 'Postavi svojstvene vrijednosti':
        method = 2
        sele.disabled = False
    return method

In [8]:
numU = 1

def main_callback2(Aw, Bw, X0w, K, L, eig1c, eig2c, eig3c, eig1o, eig2o, eig3o, u, period, selm, sele, selu, simTime, DW):
    eige = eigen_choice(sele)
    method = method_choice(selm)
    
    if method == 1:
        solc = numpy.linalg.eig(A-B[:,0:numU]*K)
        solo = numpy.linalg.eig(A-L*C)
    if method == 2:
        if eige == 0:
            K = control.acker(A, B[:,0:numU], [eig1c[0,0], eig2c[0,0], eig3c[0,0]])
            Kw.setM(K)
            L = control.place(A.T, C.T, [eig1o[0,0], eig2o[0,0], eig3o[0,0]]).T
            Lw.setM(L)
        if eige == 2:
            K = control.acker(A, B[:,0:numU], [eig3c[0,0], 
                                     numpy.complex(eig2c[0,0],eig2c[1,0]), 
                                     numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
            L = control.place(A.T, C.T, [eig3o[0,0], 
                                         numpy.complex(eig2o[0,0],eig2o[1,0]), 
                                         numpy.complex(eig2o[0,0],-eig2o[1,0])]).T
            Lw.setM(L)
            
    
    sys = control.ss(A,B,numpy.vstack((C,numpy.zeros((B.shape[1],C.shape[1])))),numpy.vstack((D,numpy.eye(B.shape[1]))))
    sysC = control.ss(numpy.zeros((1,1)),
                      numpy.zeros((1,numpy.shape(A)[0])),
                      numpy.zeros((numpy.shape(B[:,0:numU])[1],1)),
                      -K)
    
    sysE = control.ss(A-L*C,
                      numpy.hstack((L,B-L*D)),
                      numpy.eye(numpy.shape(A)[0]),
                      numpy.zeros((A.shape[0],C.shape[0]+B.shape[1])))
    
    sys_append = control.append(sys, sysE, sysC, control.ss(A,B,numpy.eye(A.shape[0]),numpy.zeros((A.shape[0],B.shape[1]))))
    Q = []
    # y in ingresso a sysE
    for i in range(C.shape[0]):
        Q.append([B.shape[1]+i+1, i+1])
    # u in ingresso a sysE
    for i in range(B.shape[1]):
        Q.append([B.shape[1]+C.shape[0]+i+1, C.shape[0]+i+1])
    # u in ingresso a sys
    for i in range(B[:,0:numU].shape[1]):
        Q.append([i+1, C.shape[0]+B.shape[1]+A.shape[0]+i+1])
    # u in ingresso al sistema che ha come uscite gli stati reali
    for i in range(B.shape[1]):
        Q.append([2*B.shape[1]+C.shape[0]+A.shape[0]+i+1, C.shape[0]+i+1])
    # xe in ingresso a sysC
    for i in range(A.shape[0]):
        Q.append([2*B.shape[1]+C.shape[0]+i+1, C.shape[0]+B.shape[1]+i+1])
        
    inputv = [i+numU+1 for i in range(B[:,numU:].shape[1])]
    outputv = [i+1 for i in range(numpy.shape(sys_append.C)[0])]
    # in order to avoid singular value exception
    try:
        sys_CL = control.connect(sys_append, Q, inputv, outputv)
    except:
        sys_CL = control.connect(sys_append, Q, inputv, outputv)
    
    dcgain = control.dcgain(sys_CL[0,0])
    gain_w2.value = dcgain
    if dcgain != 0:
        u1 = u/gain_w2.value
    else:
        print('Unaprijedno pojačanje je postavljeno na 0 i promijenjeno na 1')
        u1 = u/1
    print('Statičko pojačanje sustava u zatvorenoj petlji (od ref. signala do izlaza) je: %.5f' %dcgain)
    
    X0w1 = numpy.zeros((A.shape[0],1))
    for j in range(A.shape[0]):
        X0w1 = numpy.vstack((X0w1,X0w[j]))
    X0w1 = numpy.vstack((X0w1,numpy.zeros((A.shape[0],1))))
    if simTime != 0:
        T = numpy.linspace(0, simTime, 10000)
    else:
        T = numpy.linspace(0, 1, 10000)
      
    if selu == 'impuls': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        U1 = [0 for t in range(0,len(T))]
        U1[0] = u1
        T, yout, xout = control.forced_response(sys_CL[:,0],T,U1,X0w1)
    if selu == 'step':
        U = [u for t in range(0,len(T))]
        U1 = [u1 for t in range(0,len(T))]
        T, yout, xout = control.forced_response(sys_CL[:,0],T,U1,X0w1)
    if selu == 'sinus':
        U = u*numpy.sin(2*numpy.pi/period*T)
        U1 = u1*numpy.sin(2*numpy.pi/period*T)
        T, yout, xout = control.forced_response(sys_CL[:,0],T,U1,X0w1)
    if selu == 'Pravokutni val':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        U1 = u1*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        T, yout, xout = control.forced_response(sys_CL[:,0],T,U1,X0w1)
    # N.B. i primi 3 stati di xout sono quelli del sistema, mentre gli ultimi 3 sono quelli dell'osservatore
    
    step_info_dict = control.step_info(sys_CL[0,0],SettlingTimeThreshold=0.02,T=T)
    print('Informacije o koraku: \n\tVrijeme porasta =',step_info_dict['RiseTime'],'\n\tVrijeme smirivanja (2%) =',step_info_dict['SettlingTime'],'\n\tPrekoračenje (%)=',step_info_dict['Overshoot'])
    print('Maksimalna U vrijednost (% of 5A)=', max(abs(yout[C.shape[0]]))/(5)*100)
    
    fig = plt.figure(num='Simulation1', figsize=(14,12))
    
    fig.add_subplot(221)
    plt.title('Izlazni odziv')
    plt.ylabel('Izlaz')
    plt.plot(T,yout[0],T,U,'r--')
    plt.xlabel('$t$ [s]')
    plt.legend(['$y$','Referentni signal'])
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(222)
    plt.title('Ulaz')
    plt.ylabel('$u$')
    plt.plot(T,yout[C.shape[0]],T,[5 for i in range(len(T))],'r--')
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(223)
    plt.title('Odziv stanja')
    plt.ylabel('Stanja')
    plt.plot(T,yout[C.shape[0]+B.shape[1]+B[:,numU:].shape[1]+A.shape[0]],
             T,yout[C.shape[0]+B.shape[1]+B[:,numU:].shape[1]+A.shape[0]+1],
             T,yout[C.shape[0]+B.shape[1]+B[:,numU:].shape[1]+A.shape[0]+2])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$x_{1}$','$x_{2}$','$x_{3}$'])
    plt.grid()
    
    fig.add_subplot(224)
    plt.title('Pogreška procjene')
    plt.ylabel('Pogreška')
    plt.plot(T,yout[C.shape[0]+B.shape[1]+B[:,numU:].shape[1]+A.shape[0]]-yout[C.shape[0]+B.shape[1]],
             T,yout[C.shape[0]+B.shape[1]+B[:,numU:].shape[1]+A.shape[0]+1]-yout[C.shape[0]+B.shape[1]+1],
             T,yout[C.shape[0]+B.shape[1]+B[:,numU:].shape[1]+A.shape[0]+2]-yout[C.shape[0]+B.shape[1]+2])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$e_{1}$','$e_{2}$','$e_{3}$'])
    plt.grid()
    #plt.tight_layout()
   
alltogether2 = widgets.VBox([widgets.HBox([selm, 
                                          sele,
                                          selu]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('K:',border=3), Kw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Svojstvene vrijednosti:',border=3), 
                                          eig1c, 
                                          eig2c, 
                                          eig3c,
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('X0 est.:',border=3), X0w]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('L:',border=3), Lw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Svojstvene vrijednosti:',border=3), 
                                          eig1o, 
                                          eig2o, 
                                          eig3o,
                                          widgets.Label(' ',border=3)]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.VBox([widgets.Label('Inverzno referentno pojačanje:',border=3),
                            widgets.Label('Vrijeme simulacije [s]:',border=3)]),
                            widgets.VBox([gain_w2,simTime])]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([u, 
                                          period, 
                                          START])])
out2 = widgets.interactive_output(main_callback2, {'Aw':Aw, 'Bw':Bw, 'X0w':X0w, 'K':Kw, 'L':Lw,
                                                 'eig1c':eig1c, 'eig2c':eig2c, 'eig3c':eig3c, 'eig1o':eig1o, 'eig2o':eig2o, 'eig3o':eig3o, 
                                                 'u':u, 'period':period, 'selm':selm, 'sele':sele, 'selu':selu, 'simTime':simTime, 'DW':DW})
out2.layout.height = '870px'
display(out2, alltogether2)

Output(layout=Layout(height='870px'))